In [1]:
# import torch
# print("PyTorch version:", torch.__version__)
# print("CUDA available:", torch.cuda.is_available())
# if torch.cuda.is_available():
#     print("GPU:", torch.cuda.get_device_name(0))


In [2]:
# import zipfile
# import requests
# import os

# # Tạo thư mục lưu dữ liệu
# os.makedirs(r"C:\Users\PC\coco\images", exist_ok=True)
# os.makedirs(r"C:\Users\PC\coco\annotations", exist_ok=True)

# # Hàm tải file
# def download_file(url, save_path):
#     response = requests.get(url, stream=True)
#     with open(save_path, 'wb') as f:
#         for chunk in response.iter_content(chunk_size=8192):
#             f.write(chunk)
#     print(f"✅ Đã tải {save_path}")

# # Hàm giải nén và xóa zip
# def unzip_and_remove(zip_path, extract_to):
#     with zipfile.ZipFile(zip_path, 'r') as zip_ref:
#         zip_ref.extractall(extract_to)
#     os.remove(zip_path)
#     print(f"✅ Đã giải nén và xóa {zip_path}")

# # URLs cho COCO 2017
# urls = {
#     "train2017.zip": "http://images.cocodataset.org/zips/train2017.zip",
#     "val2017.zip": "http://images.cocodataset.org/zips/val2017.zip",
#     "annotations_trainval2017.zip": "http://images.cocodataset.org/annotations/annotations_trainval2017.zip"
# }



# print("✅ Đã cài đặt xong và tạo thư mục dữ liệu.")
# # Tải và xử lý dữ liệu
# for filename, url in urls.items():
#     download_file(url, filename)
#     extract_to = r"C:\Users\PC\coco\images" if "train" in filename or "val" in filename else r"C:\Users\PC\coco\annotations"
#     unzip_and_remove(filename, extract_to)

In [3]:
# pip install tensorflow-gpu==2.10.1

In [4]:
# import tensorflow as tf
# print("TensorFlow version:", tf.__version__)
# print("Available GPU(s):", tf.config.list_physical_devices('GPU'))

In [5]:
yaml_content = """
path: C:\\Users\\PC\\coco
train: train2017.txt
val: val2017.txt

names:
  0: person
  
kpt_shape: [17, 3] # number of keypoints, number of dims (2 for x,y or 3 for x,y,visible)
flip_idx: [0, 2, 1, 4, 3, 6, 5, 8, 7, 10, 9, 12, 11, 14, 13, 16, 15]

"""

with open(r"C:\Users\PC\new_coco-pose.yaml", "w") as f:
    f.write(yaml_content)
print("✅ Đã tạo file new_coco-pose.yaml!")

✅ Đã tạo file new_coco-pose.yaml!


In [6]:
# import json

# def convert_coco_to_yolo_keypoints(coco_json_path, images_dir, labels_dir):
#     os.makedirs(labels_dir, exist_ok=True)
#     with open(coco_json_path) as f:
#         coco = json.load(f)

#     image_id_to_filename = {img['id']: img['file_name'] for img in coco['images']}

#     for ann in coco['annotations']:
#         if ann['num_keypoints'] == 0:
#             continue  # Bỏ qua ảnh không có keypoints

#         image_id = ann['image_id']
#         bbox = ann['bbox']
#         keypoints = ann['keypoints']

#         x_center = (bbox[0] + bbox[2] / 2) / 640
#         y_center = (bbox[1] + bbox[3] / 2) / 640
#         width = bbox[2] / 640
#         height = bbox[3] / 640

#         # Chuẩn hóa keypoints
#         kp_norm = [str(kp / 640 if i % 3 != 2 else kp) for i, kp in enumerate(keypoints)]

#         label_line = f"0 {x_center} {y_center} {width} {height} {' '.join(kp_norm)}\n"
#         label_file = os.path.join(labels_dir, image_id_to_filename[image_id].replace('.jpg', '.txt'))

#         with open(label_file, 'a') as f:
#             f.write(label_line)

#     print(f"✅ Chuyển đổi xong {len(coco['annotations'])} annotations → {labels_dir}")

# # Chuyển đổi nhãn cho train và val
# convert_coco_to_yolo_keypoints(r"C:\Users\PC\coco\images\annotations\person_keypoints_train2017.json",
#                                 r"C:\Users\PC\coco\images\train2017",
#                                r"C:\Users\PC\coco\labels\train2017")

# convert_coco_to_yolo_keypoints(r"C:\Users\PC\coco\images\annotations\person_keypoints_val2017.json",
#                                r"C:\Users\PC\coco\images\val2017",
#                                r"C:\Users\PC\coco\labels\val2017")


In [7]:
%%writefile dyyolov8-pose.yaml
nc: 1 # number of classes
kpt_shape: [17, 3] # number of keypoints, number of dims (2 for x,y or 3 for x,y,visible)
scales: # model compound scaling constants, i.e. 'model=yolov8n-pose.yaml' will call yolov8-pose.yaml with scale 'n'
  # [depth, width, max_channels]
  n: [0.33, 0.25, 1024]
  s: [0.33, 0.50, 1024]
  m: [0.67, 0.75, 768]
  l: [1.00, 1.00, 512]
  x: [1.00, 1.25, 512]

# YOLOv8.0n backbone
backbone:
  # [from, repeats, module, args]
  - [-1, 1, Conv, [64, 3, 2]] # 0-P1/2
  - [-1, 1, Conv, [128, 3, 2]] # 1-P2/4
  - [-1, 3, DyC2f, [128, True]]
  - [-1, 1, Conv, [256, 3, 2]] # 3-P3/8
  - [-1, 6, DyC2f, [256, True]]
  - [-1, 1, Conv, [512, 3, 2]] # 5-P4/16
  - [-1, 6, DyC2f, [512, True]]
  - [-1, 1, Conv, [1024, 3, 2]] # 7-P5/32
  - [-1, 3, DyC2f, [1024, True]]
  - [-1, 1, SPPF, [1024, 5]] # 9

# YOLOv8.0n head
head:
  - [-1, 1, nn.Upsample, [None, 2, "nearest"]]
  - [[-1, 6], 1, Concat, [1]] # cat backbone P4
  - [-1, 3, DyC2f, [512]] # 12

  - [-1, 1, nn.Upsample, [None, 2, "nearest"]]
  - [[-1, 4], 1, Concat, [1]] # cat backbone P3
  - [-1, 3, DyC2f, [256]] # 15 (P3/8-small)

  - [-1, 1, Conv, [256, 3, 2]]
  - [[-1, 12], 1, Concat, [1]] # cat head P4
  - [-1, 3, DyC2f, [512]] # 18 (P4/16-medium)

  - [-1, 1, Conv, [512, 3, 2]]
  - [[-1, 9], 1, Concat, [1]] # cat head P5
  - [-1, 3, DyC2f, [1024]] # 21 (P5/32-large)

  - [[15, 18, 21], 1, Pose, [nc, kpt_shape]] # Pose(P3, P4, P5)

Overwriting dyyolov8-pose.yaml


In [8]:
file_path = r"C:\Users\PC\anaconda3\envs\train_env\Lib\site-packages\ultralytics\nn\modules\block.py"

# anaconda3/envs/train_env/Lib/site-packages/ultralytics/nn/modules/block.py
c2f_class_code = """

class ContextGenerationModule(nn.Module):
    def __init__(self, in_channels, reduction=16):
        super(ContextGenerationModule, self).__init__()
        reduced_channels = max(1, in_channels // reduction)

        self.avg_pool_w = nn.AdaptiveAvgPool2d((1, None))  # Eq. (2)
        self.avg_pool_h = nn.AdaptiveAvgPool2d((None, 1))  # Eq. (3)

        self.shared_fc = nn.Sequential(
            nn.Linear(in_channels, reduced_channels, bias=False),
            nn.BatchNorm1d(reduced_channels),
            nn.Hardswish()
        )

        self.fc_out = nn.Linear(reduced_channels * 2, in_channels, bias=True)  # Eq. (6)

    def forward(self, x):
        b, c, h, w = x.size()

        x_w = self.avg_pool_w(x).view(b, c, w)  # (B, C, W)
        x_h = self.avg_pool_h(x).view(b, c, h)  # (B, C, H)

        x_w = self.shared_fc(x_w.permute(0, 2, 1)).permute(0, 2, 1)  # Eq. (4)
        x_h = self.shared_fc(x_h.permute(0, 2, 1)).permute(0, 2, 1)  # Eq. (4)

        x_context = torch.cat([x_w.mean(dim=2), x_h.mean(dim=2)], dim=1)  # Eq. (5)
        kernel_weights = self.fc_out(x_context).view(b, c, 1, 1)  # Eq. (6)

        return kernel_weights

class DyC2f(nn.Module):

    def __init__(self, c1, c2, n=1, shortcut=False, g=1, e=0.5):
        super().__init__()
        self.c = int(c2 * e)  # hidden channels
        self.cv1 = Conv(c1, 2 * self.c, 1, 1)
        self.cv2 = Conv((2 + n) * self.c, c2, 1)  # optional act=FReLU(c2)
        self.cgm = ContextGenerationModule(c2, reduction=4)
        self.m = nn.ModuleList(Bottleneck(self.c, self.c, shortcut, g, k=((3, 3), (3, 3)), e=1.0) for _ in range(n))

    def forward(self, x):
        #kernel_weights = self.cgm(x)  # Dynamic kernel generation
        y = list(self.cv1(x).chunk(2, 1))
        y.extend(m(y[-1]) for m in self.m)
        return self.cv2(torch.cat(y, 1))# + kernel_weights
"""

# Append the class definition to the file
with open(file_path, "a") as f:
    f.write("\n" + c2f_class_code)

print("DyC2f class successfully appended to block.py")


DyC2f class successfully appended to block.py


In [9]:
import os

file_path = r"C:\Users\PC\anaconda3\envs\train_env\Lib\site-packages\ultralytics\nn\tasks.py"

if not os.path.exists(file_path):
    print("File does not exist.")
else:
    # Read the file contents with utf-8 encoding
    with open(file_path, 'r', encoding='utf-8') as file:
        filedata = file.read()

    # Replace the target string
    newdata = filedata.replace(" C2f,\n", " C2f, DyC2f,\n")

    # Write the file out again with utf-8 encoding
    with open(file_path, 'w', encoding='utf-8') as file:
        file.write(newdata)

    print("DyC2f class successfully appended to block.py")


DyC2f class successfully appended to block.py


In [10]:
# Define the file path
file_path = r"C:\Users\PC\anaconda3\envs\train_env\Lib\site-packages\ultralytics\nn\modules\__init__.py"

# Check if the file exists
if not os.path.isfile(file_path):
    print(f"File not found: {file_path}")
else:
    # Read the file contents
    with open(file_path, 'r') as file:
        filedata = file.read()

    # Replace the target string
    newdata = filedata.replace(" C2f,\n", " C2f, DyC2f,\n")

    # Write the modified content back to the file
    with open(file_path, 'w') as file:
        file.write(newdata)

    print("Replacement complete.")

Replacement complete.


In [11]:
# pip install torchsummary

In [12]:
import torch
import torch.nn as nn
from ultralytics import YOLO
import os
from torchsummary import summary
from ultralytics.nn.modules import C2f # Import the C2F class

In [13]:
FallDeteNet_V2 = YOLO(r"C:\Users\PC\dyyolov8n-pose.yaml")


In [14]:
import os
import torch
import pandas as pd

In [15]:
best_loss = float("inf")  # Giá trị loss tốt nhất
results = []  # Danh sách lưu kết quả từng epoch

In [16]:
def train_model(model, data_yaml, epochs=50, batch_size=128, img_size=320, device="cuda"):
    """
    Huấn luyện mô hình FallDeteNet_V2 trên COCO-Pose dataset và lưu các giá trị loss, metric chi tiết vào Google Drive.

    Args:
        model: Mô hình đã được khởi tạo từ CustomYOLOPose_v2.
        data_yaml: Đường dẫn đến file coco-pose.yaml.
        epochs: Số epoch huấn luyện.
        batch_size: Kích thước batch.
        img_size: Kích thước ảnh.
        device: Thiết bị huấn luyện (mặc định: "cuda").
    """

    global best_loss, results

    # Kiểm tra thiết bị
    device = torch.device(device if torch.cuda.is_available() else "cpu")
    model.to(device)

    # Tiến hành huấn luyện
    for epoch in range(epochs):
        print(f"\n🚀 Epoch {epoch+1}/{epochs} đang huấn luyện...")

        # Huấn luyện và lấy metrics
        metrics = model.train(
            data=data_yaml,
            epochs= epochs,  # Chạy từng epoch một để lưu kết quả sau mỗi lần
            batch=batch_size,
            imgsz=img_size,
            device=device,
            workers=6,
            name="FallDeteNet_V2",
            verbose=True,
        )

In [ ]:
train_model(FallDeteNet_V2, "new_coco-pose.yaml", epochs=100, batch_size=32, img_size=640, device="cuda")


🚀 Epoch 1/100 đang huấn luyện...
engine\trainer: task=pose, mode=train, model=C:\Users\PC\dyyolov8n-pose.yaml, data=new_coco-pose.yaml, epochs=100, time=None, patience=100, batch=32, imgsz=640, save=True, save_period=-1, cache=False, device=cuda, workers=6, project=None, name=FallDeteNet_V26, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_masks=False, embed=None, show=False, save_frames=False, save_txt=False, save_conf=False, save_crop=False, show_labels=True, show_conf=True, show_boxes=True, line_width=None, fo

train: Scanning C:\Users\PC\coco\labels\train2017.cache... 56599 images, 0 backgrounds, 0 corrupt: 10
val: Scanning C:\Users\PC\coco\labels\val2017.cache... 2346 images, 0 backgrounds, 0 corrupt: 100%|██


Plotting labels to runs\pose\FallDeteNet_V26\labels.jpg... 
optimizer: 'optimizer=auto' found, ignoring 'lr0=0.01' and 'momentum=0.937' and determining best 'optimizer', 'lr0' and 'momentum' automatically... 
optimizer: SGD(lr=0.01, momentum=0.9) with parameter groups 71 weight(decay=0.0), 89 weight(decay=0.0005), 88 bias(decay=0.0)
TensorBoard: model graph visualization added 
Image sizes 640 train, 640 val
Using 6 dataloader workers
Logging results to runs\pose\FallDeteNet_V26
Starting training for 100 epochs...

      Epoch    GPU_mem   box_loss  pose_loss  kobj_loss   cls_loss   dfl_loss  Instances       Size


      1/100      4.95G      2.888      9.389     0.6795      2.946      3.422        125        640: 
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95)     Pose(P  


                   all       2346       6352      0.398      0.318      0.298      0.121     0.0427     0.0348    0.00529   0.000922

      Epoch    GPU_mem   box_loss  pose_loss  kobj_loss   cls_loss   dfl_loss  Instances       Size


...


      Epoch    GPU_mem   box_loss  pose_loss  kobj_loss   cls_loss   dfl_loss  Instances       Size


     99/100      4.32G     0.9056      2.866     0.3151      0.706      1.152         58        640: 
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95)     Pose(P  

                   all       2346       6352      0.873       0.79      0.893       0.67      0.818      0.711      0.759      0.435



      Epoch    GPU_mem   box_loss  pose_loss  kobj_loss   cls_loss   dfl_loss  Instances       Size


    100/100      4.28G     0.9066      2.886     0.3161     0.7083      1.151         92        640: 